In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [203]:

models = ["LLaVa-OneVision-0.5b-ov", "Janus-Pro-1B", "Qwen2-VL-2B-Instruct"]
lists = ["TestingIsThereLlava.csv", "TestingJanusIsThere.csv", "TestingIsThereQwen.csv"]

In [204]:
def metrics_from_df(df):
    mode_names = {
            0: "Detector con reglas, MLLM e información",
            1: "MLLM Solo",
            2: "Detector, MLLM e informacion",
            3: "Detector con reglas y MLLM",
            4: "Detector con reglas ",
        }
    df.rename(columns={'True Event': 'Event', "True Positive": "TP", "False Positive": "FP", "False Negative": "FN", "True Negative": "TN"}, inplace=True)
    df_sum = df.groupby(["Event", "Mode"])[ ["TP", "FP", "FN", "TN"]].sum().reset_index()

    # Initialize precision, recall, accuracy with zeros
    df_sum["Precision"] = 0.0
    df_sum["Recall"] = 0.0
    df_sum["Accuracy"] = 0.0

    # Calculate only where the denominator is not zero
    mask_prec = (df_sum["TP"] + df_sum["FP"]) != 0
    df_sum.loc[mask_prec, "Precision"] = (
        df_sum.loc[mask_prec, "TP"] / (df_sum.loc[mask_prec, "TP"] + df_sum.loc[mask_prec, "FP"])
    )

    mask_recall = (df_sum["TP"] + df_sum["FN"]) != 0
    df_sum.loc[mask_recall, "Recall"] = (
        df_sum.loc[mask_recall, "TP"] / (df_sum.loc[mask_recall, "TP"] + df_sum.loc[mask_recall, "FN"])
    )

    mask_acc = (df_sum["TP"] + df_sum["FP"] + df_sum["FN"] + df_sum["TN"]) != 0
    df_sum.loc[mask_acc, "Accuracy"] = (
        (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "TN"]) /
        (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "FP"] + df_sum.loc[mask_acc, "FN"] + df_sum.loc[mask_acc, "TN"])
    )
    df_sum.drop(columns=['TP', 'FP', 'FN', 'TN'], inplace=True)
    df_sum.drop(columns=['Event'], inplace=True)
    df_sum=df_sum.groupby(["Mode"]).mean()
    df_sum.rename(index=mode_names, inplace=True)
    df_sum['F1'] = 2 * df_sum['Precision'].astype(float) * df_sum['Recall'].astype(float) / (df_sum['Precision'].astype(float) + df_sum['Recall'].astype(float))
    df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero

    # Create a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()

    # Adjust "Process time" based on "Duration" and video naming convention
    df_copy["Process time"] = df_copy["Process time"] / df_copy["Duration"]
    df_copy.loc[df_copy["Name"].str.contains("_1.mp4"), "Process time"] = 30.0 / df_copy["Process time"]
    df_copy.loc[~df_copy["Name"].str.contains("_1.mp4"), "Process time"] = 25.0 / df_copy["Process time"]
    # Calculate FPS metrics
    fps_values = df_copy.groupby(['Event',"Mode"])["Process time"].mean().reset_index().drop(columns=['Event'])
    fps_values =fps_values.groupby(["Mode"]).mean().reset_index()
    fps_values.rename(index=mode_names, inplace=True)
    df_sum["FPS"] = fps_values["Process time"].values
    return df_sum

In [205]:
for i in range(len(lists)):
    df = pd.read_csv(f"/home/ubuntu/Tesis/Results/Tesis/SelectionOfModel/{lists[i]}")
    df = df[df["Mode"] == 1]
    metrics=metrics_from_df(df)

    print(f'{models[i]}\n\n', metrics)

LLaVa-OneVision-0.5b-ov

            Precision   Recall  Accuracy        F1        FPS
Mode                                                        
MLLM Solo    0.65231  0.18217  0.650604  0.284803  13.406538
Janus-Pro-1B

            Precision    Recall  Accuracy       F1       FPS
Mode                                                       
MLLM Solo    0.59301  0.708112  0.606514  0.64547  6.088385
Qwen2-VL-2B-Instruct

            Precision    Recall  Accuracy        F1       FPS
Mode                                                        
MLLM Solo   0.751149  0.335669  0.701432  0.463992  4.268769


/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [206]:
import os 
#Several promts
rute = "/home/ubuntu/Tesis/Results/temp"
files = os.listdir(rute)
file_names = {
        "Is": "is 'event' in the video?",
        "Tell_IsThere": "tell me if in the video is there 'event'?",
        "Tell_Is": "tell me if is 'event' in the video? ",
        "Does": "does the video contain 'event'?",
        "IsThere": "is there 'event'?",
        "Confirm": "confirm if the video contain 'event'",
    }
for i in range(len(files)):
    df=pd.read_csv(f"{rute}/{files[i]}")
    df = df[df["Mode"] == 1]
    metrics=metrics_from_df(df)
    print(file_names[files[i].split(".")[0].split("TestingJanus")[-1]], metrics, '\n')



is 'event' in the video?            Precision    Recall  Accuracy       F1       FPS
Mode                                                       
MLLM Solo   0.534005  0.923505  0.564908  0.67671  5.956193 

tell me if in the video is there 'event'?            Precision    Recall  Accuracy       F1       FPS
Mode                                                       
MLLM Solo   0.672942  0.545097  0.652417  0.60231  6.212656 

tell me if is 'event' in the video?             Precision    Recall  Accuracy        F1       FPS
Mode                                                        
MLLM Solo   0.690864  0.589746  0.637823  0.636313  5.831522 

does the video contain 'event'?            Precision   Recall  Accuracy        F1       FPS
Mode                                                       
MLLM Solo   0.558552  0.90155  0.592883  0.689763  5.849027 

confirm if the video contain 'event'            Precision    Recall  Accuracy        F1       FPS
Mode                               

/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [207]:
#Prompts

#prompt simple
file='TestingJanusAllOnlyTrue.csv'
rute='/home/ubuntu/Tesis/Results/Tesis/PerformanceOldPrompt'
df = pd.read_csv(f'{rute}/{file}')
df = df[(df['Mode'] == 0) | (df['Mode'] == 2)]
metrics = metrics_from_df(df)
print(f'Prompt simple\n\n', metrics)
#prompt with localization
file='TestingJanusPrompts.csv'
rute='/home/ubuntu/Tesis/Results/Tesis/PerformanceNewPrompt'
df = pd.read_csv(f'{rute}/{file}')
df = df[(df['Mode'] == 0) | (df['Mode'] == 2)]
metrics = metrics_from_df(df)
print(f'\nPrompt with localization\n\n', metrics)

Prompt simple

                                          Precision    Recall  Accuracy  \
Mode                                                                     
Detector con reglas, MLLM e información   0.749284  0.922299  0.718747   
Detector, MLLM e informacion              0.483267  0.912121  0.517600   

                                               F1        FPS  
Mode                                                          
Detector con reglas, MLLM e información  0.826837  18.924638  
Detector, MLLM e informacion             0.631793   5.139875  

Prompt with localization

                                          Precision    Recall  Accuracy  \
Mode                                                                     
Detector con reglas, MLLM e información   0.745118  0.999266  0.745106   
Detector, MLLM e informacion              0.468831  0.988042  0.491493   

                                               F1        FPS  
Mode                                           

/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [208]:
#Todos los videos arquitectura
df = pd.read_csv(f"/home/ubuntu/Tesis/Results/Tesis/PerformanceNewPrompt/TestJanusAll5.csv")
metrics=metrics_from_df(df)
print('\nAll\n', metrics)
df_chad=df.loc[df["Name"].str.contains("_1.mp4")] 
metrics=metrics_from_df(df_chad)
print('\nCHAD\n', metrics)
df_others=df.loc[~df["Name"].str.contains("_1.mp4")] 
metrics=metrics_from_df(df_others)
print('\nOthers\n', metrics) 


All
                                          Precision    Recall  Accuracy  \
Mode                                                                     
Detector con reglas, MLLM e información   0.660884  0.999204  0.661164   
MLLM Solo                                 0.538725  0.771577  0.612019   
Detector, MLLM e informacion              0.453356  0.973084  0.485356   
Detector con reglas y MLLM                0.741165  0.801464  0.721844   
Detector con reglas                       0.663671  0.320521  0.606194   

                                               F1        FPS  
Mode                                                          
Detector con reglas, MLLM e información  0.795570  17.984386  
MLLM Solo                                0.634461   6.224142  
Detector, MLLM e informacion             0.618537   4.605602  
Detector con reglas y MLLM               0.770136  18.997876  
Detector con reglas                      0.432274  28.861179  

CHAD
                            

/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
/tmp/ipykernel_7600/3718373719.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'True Event': 'Event', "True Positive": "TP", "False Positive": "FP", "False Negative": "FN", "

In [213]:
#CLIP
mode_names = {
        'Detector con reglas, MLLM e información': "CLIP y Reglas",
        'MLLM Solo': "CLIP y Reglas con MLLM",
    }
df = pd.read_csv(f"/home/ubuntu/Tesis/Results/Tesis/CLIP/TestingDevStorage.csv")
metrics=metrics_from_df(df)
metrics.rename(index=mode_names, inplace=True)
print('\nAll\n', metrics)
df_chad=df.loc[df["Name"].str.contains("_1.mp4")] 
metrics=metrics_from_df(df_chad)
metrics.rename(index=mode_names, inplace=True)
print('\nCHAD\n', metrics)
df_others=df.loc[~df["Name"].str.contains("_1.mp4")] 
metrics=metrics_from_df(df_others)
metrics.rename(index=mode_names, inplace=True)
print('\nOthers\n', metrics) 



All
                         Precision    Recall  Accuracy        F1        FPS
Mode                                                                      
CLIP y Reglas            0.588286  0.078782  0.566432  0.138956  23.066971
CLIP y Reglas con MLLM   0.534555  0.266702  0.540719  0.355858  13.479743

CHAD
                         Precision    Recall  Accuracy        F1        FPS
Mode                                                                      
CLIP y Reglas            0.500774  0.106057  0.549408  0.175042  23.024287
CLIP y Reglas con MLLM   0.524620  0.255013  0.487275  0.343200  13.362610

Others
                         Precision    Recall  Accuracy        F1        FPS
Mode                                                                      
CLIP y Reglas            0.544793  0.073359  0.588800  0.129307  23.238072
CLIP y Reglas con MLLM   0.455319  0.276110  0.584842  0.343760  12.762183


/tmp/ipykernel_7600/3718373719.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
/tmp/ipykernel_7600/3718373719.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'True Event': 'Event', "True Positive": "TP", "False Positive": "FP", "False Negative": "FN", "